In [1]:
from j2v import mag
from j2v import w2v
from j2v import predict

import numpy as np
import pandas as pd
import random
from collections import defaultdict
import matplotlib.pyplot as plt
%matplotlib inline
import csv
import networkx as nx

Using Theano backend.


In [ ]:
# execute only once

pfile = 'Papers_reduced.csv'
rfile = 'PaperReferences_clean.csv'

out_pfile = 'Papers_1950_2000_J.csv'
out_rfile = 'PaperReferences_1950_2000_J.csv'

mag.filter_paper_year_jid(pfile, out_pfile, [1950, 2000], delimiter = ',')
mag.filter_refs(out_pfile, rfile, out_rfile, pdelimiter = ',', rdelimiter = ',')

In [ ]:
def dict_to_csv(csv_file, csv_columns, mdict):
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        # writer.writeheader()
        for key, value in mdict.items():
            writer.writerow({csv_columns[0]: key, csv_columns[1]: value})

### PageRank of papers in the LCC

In [ ]:
paper_file = 'Papers_1950_2000_J.csv'
ref_file = "PaperReferences_1950_2000_J.csv"

In [5]:
g = nx.read_edgelist(mag.get_path(ref_file), delimiter=",", create_using=nx.DiGraph(), nodetype=str, edgetype=int)
print(nx.info(g))
pr = nx.pagerank(g, alpha=0.85)

Name: 
Type: DiGraph
Number of nodes: 11453375
Number of edges: 71843485
Average in degree:   6.2727
Average out degree:   6.2727


In [ ]:
g = nx.read_edgelist(mag.get_path(ref_file), delimiter=",", create_using=nx.DiGraph(), nodetype=str, edgetype=int)
print(nx.info(g))
pr = nx.pagerank(g, alpha=0.85)

In [ ]:
dict_to_csv(mag.get_path('pr_pid_1950_2000.csv'), ['pid', 'pr_value'], pr)

In [ ]:
del g

### Shuffle citations in the LCC of the paper citation network

In [ ]:
pids_lcc = set()
for line in mag.yield_one_line('pr_pid_1950_2000.csv', delimiter=','):
    pid, pr = line
    pids_lcc.add(pid)

In [ ]:
pid_year = {}
for line in mag.yield_one_line(paper_file, delimiter=","):
    pid, year, jid, cid = line
    pid_year[pid] = year

In [ ]:
len(pid_year)

#### Preserve the # citations between each pair of years

In [20]:
lcc_year_pids = defaultdict(set)
lcc_pairs = []
for line in mag.yield_one_line(ref_file, delimiter=','):
    src, des = line
    if src in pids_lcc and des in pids_lcc:
        lcc_year_pids[pid_year[src]].add(src)
        lcc_year_pids[pid_year[des]].add(des)
        lcc_pairs.append((src, des))

for year, pids in lcc_year_pids.items():
    lcc_year_pids[year] = list(pids)

processing /l/nx/data/haopeng/j2v/PaperReferences_1950_2000_J.csv...
processed 1000000 lines...
processed 2000000 lines...
processed 3000000 lines...
processed 4000000 lines...
processed 5000000 lines...
processed 6000000 lines...
processed 7000000 lines...
processed 8000000 lines...
processed 9000000 lines...
processed 10000000 lines...
processed 11000000 lines...
processed 12000000 lines...
processed 13000000 lines...
processed 14000000 lines...
processed 15000000 lines...
processed 16000000 lines...
processed 17000000 lines...
processed 18000000 lines...
processed 19000000 lines...
processed 20000000 lines...
processed 21000000 lines...
processed 22000000 lines...
processed 23000000 lines...
processed 24000000 lines...
processed 25000000 lines...
processed 26000000 lines...
processed 27000000 lines...
processed 28000000 lines...
processed 29000000 lines...
processed 30000000 lines...
processed 31000000 lines...
processed 32000000 lines...
processed 33000000 lines...
processed 340000

In [33]:
len(lcc_pairs)

28098938

In [22]:
shuffle_citations = []
count = 0
for src, des in lcc_pairs:
    new_s = random.choice(lcc_year_pids[pid_year[src]])
    new_d = random.choice(lcc_year_pids[pid_year[des]])
    shuffle_citations.append((new_s, new_d))
    count += 1

In [ ]:
g = nx.DiGraph()
g.add_edges_from(shuffle_citations)
print(nx.info(g))
pr = nx.pagerank(g, alpha=0.85)

Name: 
Type: DiGraph
Number of nodes: 11453158
Number of edges: 71843433
Average in degree:   6.2728
Average out degree:   6.2728


In [25]:
len(pr)

11453158

In [26]:
dict_to_csv(mag.get_path('pr_random_preserve_year_1950_2000.csv'), ['pid', 'pr_value'], pr)